## Teil 2

Matrikelnummern: 1679412

Folgende Regressionsarten werden verwendet um eine vorhersage das SalePrices zu erstellen:
* Lineare Regression
* RandomForest
* ???

In [203]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import make_column_transformer

In [204]:
# read a specific column from a csv file
def read_data(path):
    df = pd.read_csv(path, sep=";")
    return df

# scaling numerical values and turning objects into numerical values
def scale_and_convert_columns(df, model):
    categoricalCols = []
    for i in range(len(df.columns)):
        if (df.dtypes[i] == object):
            categoricalCols.append(i)
    numericalCols = []
    for i in range(len(df.columns)):
        if (df.dtypes[i] == 'int64' ):
            numericalCols.append(i)        
            
    print(categoricalCols)
    print(numericalCols)
    
    preprocessor = make_column_transformer((categoricalCols, OneHotEncoder(handle_unknown='ignore')),
                                            (numericalCols, RobustScaler()))
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])
    return pipe
    #return pd.get_dummies(df, columns=categoricalCols)

# calculates R2, MSE, RMSE, MAPE and MAX* for the given regression model
def  calc_regression(model):
    df = read_data('SetFiltered.csv')
    pipe = scale_and_convert_columns(df.drop('SalePrice', axis=1), model)
    
    x = df.drop('SalePrice', axis=1).values #dimensions
    y = df['SalePrice'].values #prediction
    
    # neg_root_mean_squared_error and neg_median_absolute_error somehow don't work
    scores = cross_validate(pipe, x,y, cv=5, scoring=['r2', 'neg_mean_squared_error', 
                                                      'neg_root_mean_squared_error', 'neg_median_absolute_error', 
                                                      'max_error' ])
    return {'R2': scores['r2'].mean(), 'MSE': scores['neg_mean_squared_error'], 
            'RMSE': np.sqrt(scores['neg_root_mean_squared_error']), 'MAPE': scores['neg_median_absolute_error'], 
            'MAX': scores['max_error'].mean()*-1}

#calculate the score for each model
regressionTypes = [LinearRegression()]
results = []
for r in regressionTypes:
    results.append(calc_regression(r))

df = pd.DataFrame(data=results, columns=['Regression Type', 'R2', 'MSE', 'RMSE', 'MAPE', 'MAX'])
df.head()


[0, 2, 3, 8, 10, 11]
[1, 4, 5, 6, 7, 9, 12, 13, 14, 15]


E:\Programme\Python\Python36\lib\site-packages\sklearn\compose\_column_transformer.py:778: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


ValueError: 'neg_root_mean_squared_error' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.